In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_543992/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.8)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_9_23,0.974458,0.583817,0.850893,0.967406,0.935469,0.009874,0.247064,0.033055,0.019133,0.026094,0.033467,0.099367,0.954390,0.100165,119.235748,275.231252,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
1,model_1_9_17,0.974363,0.565312,0.861925,0.970289,0.940586,0.009911,0.258050,0.030609,0.017440,0.024025,0.035785,0.099552,0.954220,0.100352,119.228289,275.223794,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
2,model_1_9_18,0.974294,0.565699,0.859179,0.970328,0.939861,0.009937,0.257820,0.031218,0.017417,0.024318,0.035416,0.099686,0.954097,0.100486,119.222937,275.218442,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
3,model_1_9_19,0.974188,0.570216,0.857302,0.969567,0.938794,0.009978,0.255138,0.031634,0.017864,0.024749,0.034966,0.099892,0.953907,0.100694,119.214675,275.210180,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
4,model_1_9_24,0.974172,0.583657,0.845516,0.966518,0.933351,0.009984,0.247159,0.034247,0.019654,0.026950,0.033147,0.099922,0.953879,0.100725,119.213464,275.208968,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,model_1_6_17,0.515232,-1.041280,0.989421,-2.583779,-0.395983,0.187400,1.211792,0.005654,1.216093,0.610873,0.054322,0.432897,0.134342,0.436374,113.349024,269.344528,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
219,model_1_6_13,0.471832,-1.859083,0.989228,-2.903341,-0.520003,0.204177,1.697275,0.005758,1.324530,0.665144,0.054534,0.451859,0.056844,0.455489,113.177538,269.173043,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
220,model_1_6_16,0.469963,-1.213237,0.989358,-2.923327,-0.527673,0.204899,1.313873,0.005688,1.331312,0.668500,0.055038,0.452658,0.053506,0.456294,113.170473,269.165978,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
221,model_1_6_15,0.404221,-1.504614,0.989290,-3.418000,-0.719512,0.230314,1.486847,0.005725,1.499171,0.752448,0.055908,0.479910,-0.063891,0.483765,112.936626,268.932131,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
